# Scenario 1: A cross-functional team with one data scientist workign on an ML model

- MLflow setup:
    - Tracking server: yes, local server
    - Backend store: sqlite database
    - Artifacts store: local filesystem

- The experiments can be explored locally by accessing the local tracking server.

- To run this example you need to launch mlflow server locally by running the following command in your terminal  
    `mlflow server --backend-store-uri sqlite:///backend.db`

- By default, artifacts will be stored in ./mlartifacts. If you want to specify a directory, need to configure while setting backend-store-uri  
    `mlflow server --backend-store-uri sqlite:///backend.db --default-artifact-root ./artifacts_local`

In [2]:
import mlflow

mlflow.set_tracking_uri("http://127.0.0.1:5000")

In [5]:
print(f"tracking URI: '{mlflow.get_tracking_uri()}'")

tracking URI: 'http://127.0.0.1:5000'


## Creating an experiment and logging a new run

In [6]:
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score

mlflow.set_experiment("my-experiment-1")    #this will create and experiment if not exists. If exists, will append information of new run in it

with mlflow.start_run():   #starting a new run
    X,y=load_iris(return_X_y=True)     

    params= {"C": 0.1, "random_state": 42}
    mlflow.log_params(params)

    lr=LogisticRegression(**params).fit(X,y)
    y_pred=lr.predict(X)
    mlflow.log_metric("accuracy", accuracy_score(y,y_pred))

    mlflow.sklearn.log_model(lr, artifact_path="models")
    print(f"Default artifact URI: '{mlflow.get_artifact_uri()}'")



2025/03/11 06:53:12 INFO mlflow.tracking.fluent: Experiment with name 'my-experiment-1' does not exist. Creating a new experiment.
2025/03/11 06:53:14 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Default artifact URI: '/Users/mahmudhasan/Desktop/others/personal_projects/mlops_zoomcamp_solution/02-experiment-tracking/running-mlflow-examples/artifacts_local/1/61ab95351c8147d997e5c72f6fa5e29f/artifacts'
🏃 View run zealous-wren-550 at: http://127.0.0.1:5000/#/experiments/1/runs/61ab95351c8147d997e5c72f6fa5e29f
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1


## Interacting with the model registry

In [5]:
from mlflow.tracking import MlflowClient

client=MlflowClient("http://127.0.0.1:5000")

In [3]:
#register a model
run_id='61ab95351c8147d997e5c72f6fa5e29f'
mlflow.register_model(
    model_uri=f"runs:/{run_id}/models",
    name='iris-classifier'
)

Successfully registered model 'iris-classifier'.
2025/03/11 07:18:27 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: iris-classifier, version 1
Created version '1' of model 'iris-classifier'.


<ModelVersion: aliases=[], creation_timestamp=1741655907919, current_stage='None', description='', last_updated_timestamp=1741655907919, name='iris-classifier', run_id='61ab95351c8147d997e5c72f6fa5e29f', run_link='', source='/Users/mahmudhasan/Desktop/others/personal_projects/mlops_zoomcamp_solution/02-experiment-tracking/running-mlflow-examples/artifacts_local/1/61ab95351c8147d997e5c72f6fa5e29f/artifacts/models', status='READY', status_message='', tags={}, user_id='', version='1'>

In [7]:
client.get_registered_model(name="iris-classifier")

<RegisteredModel: aliases={}, creation_timestamp=1741655907863, description='', last_updated_timestamp=1741655907919, latest_versions=[<ModelVersion: aliases=[], creation_timestamp=1741655907919, current_stage='None', description='', last_updated_timestamp=1741655907919, name='iris-classifier', run_id='61ab95351c8147d997e5c72f6fa5e29f', run_link='', source='/Users/mahmudhasan/Desktop/others/personal_projects/mlops_zoomcamp_solution/02-experiment-tracking/running-mlflow-examples/artifacts_local/1/61ab95351c8147d997e5c72f6fa5e29f/artifacts/models', status='READY', status_message='', tags={}, user_id='', version='1'>], name='iris-classifier', tags={}>